# Predicting Pulsars
### Stellar Classification With Machine Learning

# Table of Contents
#### Data Science Workflow
An introduction and summary of the approach I took to complete this project
#### Business Case and Business Value  
Background  
Key question asked about the dataset  
#### Obtaining The Data
Sourcing the data  
Importing the CSV  
#### Scrubbing The Data
Explore potential missing values  
Ensure columns are of the correct data types  
Copy the dataset to later normalize  
#### Exploring The Data
Use basic plots to visually examine relationships within the dataset  
Exploration and familiarization with the dataset through statistical analysis  
Detecting potential skews  
Investigating possible multicollinearity   
#### Modeling The Data
Train and test split the data  
Create a baseline model to work up from  
Model the non-transformed data and review results   
Model the normalized & transformed data and compare results to the non-transformed    
Create multiple types of models to see what is the most accurate, efficient, and informative   
Tune hyperparameters to find the ideal parameters for best performing model  
#### Interpreting The Data
Compare the models created in the last step and weigh the pros & cons of each type  
Review the findings table to see which model is best, depending on our goals(accuracy, computational efficiency, etc)  
#### Conclusion
Expand upon our model review from the interpretation stage  
Conclusion and opinion on my ideal model    
Potential future work  
References

## Data Science Workflow
##### For this project I am going to use the OSEMiN process
##### (Obtain, Scrub, Explore, Model, Interpret) 

- **Obtain** - This part was simple for this project, my dataset is Dr Robert Lyon's Predicting a Pulsar Star dataset from Kaggle: (https://www.kaggle.com/pavanraj159/predicting-a-pulsar-star/data)
- **Scrub** - The objective here is to identify errors, potential missing or corrupted fields in the data, and to clean up the dataset by discarding, replacing, or filling missing values and errors. 
- **Explore** - This is largely what EDA is. The goal in this phase is to understand patterns within our dataset by visualizing and testing to examine relationships in the data. 
- **Model** - This step encompases the higher level pre-processing such as normalization and synthetic oversampling of the minority class. After that we're ready to train some models to be able to give us some accurate predictive power to make intelligent business decisions. There are multiple steps of evaluation and refining of the models to ensure they're as accurate and unbiased as possible. 
- **Interpret** - In the final step, we examine our findings developed by our modeling and indentify important insights. As more data becomes available, this cycle can repeat and continue in a loop to build a faster, less biased, and more accurate model.
 
Below you can follow the different stages of the EDA and modeling process by scrolling to the respective title.

## Business Case
#### Background
For this project, I assumed the role of a Data Scientist working with an astronomical research agency to train a machine learning model to classify whether a star is a pulsar or not.  
A pulsar is a rotating, magnetized Neutron star that emits radiation(mostly radio, X-ray, and gamma wavelength) from its poles. Other than black holes, pulsars are the most dense objects in the known universe. Pulsars rotate with a very short and precise period which vary from star to star, however due to Radio Frequency Interference and background radiation noise, there are many false-positives when attempting to detect pulsars.  
For further reading on pulsars visit the Wikipedia page [here](https://en.wikipedia.org/wiki/Pulsar). 

#### Overall Goal:  
The goal is to train a machine learning model to classify pulsar candidate stars as either a true pulsar, or a false positive. 
The overall goal is model accuracy, as computational cost is not of concern in this application.  


## Business Value  
### Why is this important?
#### Labor
In the most basic way, the ability to effectively train a machine learning algorithm to accurately classify a pulsar from a false positive would free up human assets to be able to complete other tasks. Astronomers are highly educated, and therefore, generally well paid. Freeing up these monetary assets allows them to be productive elsewhere.  
#### Cutting edge scientific discovery
Pulsars prove to be very helpful for physicists and astronomers for a variety of other reasons. Being able to quickly and effectively identify pulsars would allow more of them to be discovered and used for other types of experiments.  
Research on Pulsars has already yeilded two Nobel Peace Prizes. Pulsar research is at the tip of the spear in humanity's quest to understand gravitational waves and relativistic physics.  
#### Gravitational waves
Observing the precisely timed pulses from various pulsars scattered across the galaxy, astronomers aim to measure the slight variations in their signals which are thought to be caused by gravitational waves. Pulsar timing arrays are being used to uncover secrets of the early universe by hopefully confirming the observation of gravitational waves caused by pairs of supermassive black holes in the early universe, multidimensional strings, quasars, and other possible events from the first few moments following the Big Bang.  
#### Nuclear physics  
With pulsars being very small stellar objects(~10km radius) but having an average mass of ~1.4 solar masses, they are incredibly dense. In fact, the density of a pulsar is several times greater than the density of atomic nuclei. There is little understood about the physics of what goes on inside a pulsar and therefore are an incredibly interesting phenomena for physicists. 
#### Mapping interstellar space
Due to the regularity of the emissions from pulsars, they can be used to triangulate position in the galaxy. Humanity has included pulsar maps on the two Pioneer plaques in addition to the Voyager Golden Record. These maps use 14 pulsars, and the position of the Sun to show where we are on the galactic plane. This form of navigation could prove to be very useful as Humanity progresses to a Type II civilization(see: [Kardashev Scale](https://en.wikipedia.org/wiki/Kardashev_scale)).
#### Measuring time
Again, using the regularity of a pulsar's pulses one can measure the passing of time, specifically establishing Ephemeris time which is free of the irregular constraints related to fluctuating mean solar time. Although pulsars are very precise and regular, the interference in the readings make this form of time measurement slightly less accurate and dependable when compared to an atomic clock - this however may change as our radio telescopes become more advanced.  

# Obtain the Data  
For this project, I used Dr Robert Lyon's Predicting a Pulsar Star dataset. Also included in this step will be the importing of any libraries I'll need throughout all the steps of my analysis and modeling. 

In [ ]:
# import necessary tools
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
sns.set(style='darkgrid')

import itertools
import math
import time

import statsmodels.formula.api as smf
import xgboost as xgb
from xgboost import XGBClassifier
from scipy import stats
from sklearn import svm
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, \
    roc_curve, auc, mean_squared_error, roc_auc_score, recall_score, precision_score, \
    plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, \
    GradientBoostingClassifier, BaggingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('../input/pulsar-star/pulsar_stars.csv')
data.head()

In [ ]:
# There are 17,898 pulsar candidates within this dataset. 
# 8 columns of descriptive data about each row/star and one column of categorical data
data.shape

# Explaining the Features
The dataset describes 17,898 potential pulsar stars. There are 8 continuous variables describing the stars, and a class variable indicating whether or not the stars are actually a pulsar or not. 

To understand what these variables mean, we'll need to look into some terminology related to the classification of pulsars.  

**Integrated Pulse Profile:**  
Pulsars are distinguished by their integrated pulse profile which is a representation of its periodic pulsation signal. It is a visual representation of the pulsars emission signals with respect to its rotational period. Here's some examples of integrated pulse profiles:[1] ![Integrated Pulse Profiles](Images/Pulsar_IPP1.png)  

**DM-SNR Curve:**  
DM = dispersion measure. SNR = spectral signal-to-noise ratio.  
Dispersion measurement is the spread in frequency of the pulsar's emission after traveling through space. Free electrons and ionization within the interstellar medium affects the radiation in two ways. Due to the dispersive nature of interstellar plasma, higher-frequency waves traverse these distances more quickly than lower-frequency radio waves. The resulting delay is referred to as the dispersion measure. In other words, the DM is the "total column density of free electrons between the observer and the pulsar".  
The spectral SNR is the signal to noise ratio of all DM signal measurements taken.  
This is a common way to differentiate and classify candidate stars. Pulsar timing is exact, like that of a delta function, but radio telescopes pickup a spread centered around the peak value. This measurement is represented by the DM value, of which there is a spread from multiple measurements peaked around some DM value. You then take your DM signals and compare them as a signal-to-noise ratio for each candidate. 

- **Mean of the itegrated profile** - A mean taken from the respective pulsar's IPP.  
- **Standard deviation of the integrated profile** - The standard deviation of a pulsar's IPP.  
- **Excess kurtosis of the integrated profile** - Kurtosis measures the "fatness" of the tails of a given distribution. Positive excess kurtosis means a distribution has larger tails than a normal distribution. This means there is a less accurate distribution and may point to more noise.  
- **Skewness of the integrated profile** - Skewness measures the asymettry of a distribution. In this context, if a candidate star's IPP has a negative skew, the signal is strong initially but then tapers off quickly. If a candidate star has a positive skew, the signal is first small before peaking to the right side of the distribution.  
- **Mean of the DM-SNR curve** - This is the mean value of all dispersion measurments of a given star's signal to noise ratio.  
- **Standard deviation of the DM-SNR curve** - The standard deviation of the DM-SNR curve shows how much variation is found in the measurements of a candidate star. The higher the standard deviation could suggest there is more noise and/or interference affecting the measurements.  
- **Excess kurtosis of the DM-SNR curve** - High levels of DM-SNR kurtosis suggests more noise in an observation. 
- **Skewness of the DM-SNR curve** - A highly skewed DM-SNR curve means that there's a high amount of interference in the observation which could mean that there's a lot of ISM noise in between the candidate star and the radio telescope recording the signals.  
- **target_class** - Is this candidate a pulsar?(1:yes, 0:no)


# Scrub the Data
Now that my data has been imported, the focus will shift to preprocessing the data to be able to give us some accurate models. This includes but not limited to identifying the removing nulls, appropriately dealing with outliers, normalizing the data, and accounting for missing or improperly entered data.

In [ ]:
data.describe()

In [ ]:
# null value check
data.isna().any()

In [ ]:
data.info()

Fortunate for me, the dataset is pretty complete and clean from the source. There are no missing values, incorrect data types, or other data entry errors. Before moving on to the exploration, I'm going to clone this dataset so I can work off of a normalized and transformed set, and keep one free of normalization and transformation. 

In [ ]:
# TN(Transformed/Normalized)data
TNdata = data.copy()
TNdata.head()

# Explore the Data
In this step I'll use some basic visualizations to begin to explore relations in the dataset. 
I will further look into the dataset using visualizations and statistical analysis to gain a higher understanding of the distribution, possible multicollinearity, and other preparatory tasks to ensure I will be able to train a reliable model with the dataset.

### Target_class counts

In [ ]:
# First I want to see how many instances of each class we have in the dataset 
plt.figure(figsize=(12,12))
ax = sns.countplot(y = data.target_class, palette=["red", "green"],
                  linewidth=1, edgecolor="black")
for i, j in enumerate(data["target_class"].value_counts().values):
    ax.text(.75, i, j, fontsize=26)
plt.title("Count for target_class in dataset");

Only 9% of our target_class are actual pulsars, leaving 91% of the candidate stars in the false-positive category. 

### Describing the dataset, visualized through a heatmap

In [ ]:
# I'll use describe() again here but in a visualization to make it a bit easier to understand
plt.figure(figsize=(12,12))
ax = sns.heatmap(data.describe().transpose(),
           annot=True, linecolor="black",
            linewidth=1, square=True, vmin=0, vmax=1000, 
                 cmap=sns.color_palette("Set2"))
plt.title("Description Heatmap")
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.show()

# unfortunately due to an issue with the current release of matplotlib, the top
# and bottom rows are cut in half without using the workaround code in lines 7-8

### Correlation & multicollinearity heatmap for the features 

In [ ]:
# Next I'll look at possible multicollinearity among the features
correlation = data.corr()
plt.figure(figsize=(12,12))
ax = sns.heatmap(correlation, annot=True, 
                cmap=sns.color_palette("coolwarm", 7),
                linewidth=1, linecolor="white")
plt.title("Collinearity Between Features")
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.show()

In [ ]:
data.columns

### Comparison of mean and standard deviation between attributes for target_class

In [ ]:
# Comparison of mean and standard deviation between attributes for target_class
compareMean = data.groupby("target_class")[[' Mean of the integrated profile', 
                                            ' Standard deviation of the integrated profile', 
                                            ' Excess kurtosis of the integrated profile', 
                                            ' Skewness of the integrated profile',
                                            ' Mean of the DM-SNR curve', 
                                            ' Standard deviation of the DM-SNR curve', 
                                            ' Excess kurtosis of the DM-SNR curve',
                                        ' Skewness of the DM-SNR curve']].mean().reset_index()

compareMean = compareMean.drop("target_class", axis=1)
compareMean = compareMean.transpose().reset_index()
compareMean = compareMean.rename(columns={"index":"Features", 0:"Not a Pulsar", 1:"Pulsar"})

plt.figure(figsize=(15,15))
plt.subplot(211)
sns.pointplot(x="Features", y="Not a Pulsar", data=compareMean, color="red")
sns.pointplot(x="Features", y="Pulsar", data=compareMean, color="blue")
plt.xticks(rotation=75)
plt.xlabel("")
plt.grid(True, alpha=0.5)
plt.title("Comparing Mean of Attributes for Target Class")

compareStd = data.groupby("target_class")[[' Mean of the integrated profile', 
                                            ' Standard deviation of the integrated profile', 
                                            ' Excess kurtosis of the integrated profile', 
                                            ' Skewness of the integrated profile',
                                            ' Mean of the DM-SNR curve', 
                                            ' Standard deviation of the DM-SNR curve', 
                                            ' Excess kurtosis of the DM-SNR curve',
                                        ' Skewness of the DM-SNR curve']].std().reset_index()

compareStd = compareStd.drop("target_class", axis=1)
compareStd = compareStd.transpose().reset_index()
compareStd = compareStd.rename(columns={'index':"Features", 0:"Not a Pulsar", 1:"Pulsar"})
plt.subplot(212)
sns.pointplot(x="Features", y="Not a Pulsar", data=compareStd, color="red")
sns.pointplot(x="Features", y="Pulsar", data=compareStd, color="blue")
plt.xticks(rotation=75)
plt.grid(True, alpha=0.5)
plt.title("Comparing Standard Deviation of Attributes for Target Class")
plt.subplots_adjust(hspace = 1)
print("Blue = Pulsar, Red = Not a Pulsar")
plt.show();

### Plotted distributions of features within the dataset

In [ ]:
columns = [' Mean of the integrated profile', ' Standard deviation of the integrated profile',
           ' Excess kurtosis of the integrated profile', ' Skewness of the integrated profile',
            ' Mean of the DM-SNR curve', ' Standard deviation of the DM-SNR curve', 
           ' Excess kurtosis of the DM-SNR curve', ' Skewness of the DM-SNR curve']
length  = len(columns)
colors  = ["r","g","b","m","y","c","k","orange"] 

plt.figure(figsize=(13,20))
for i,j,k in itertools.zip_longest(columns,range(length),colors):
    plt.subplot(length/2,length/4,j+1)
    sns.distplot(data[i],color=k)
    plt.title(i)
    plt.subplots_adjust(hspace = .3)
    plt.axvline(data[i].mean(),color = "k",linestyle="dashed",label="MEAN")
    plt.axvline(data[i].std(),color = "b",linestyle="dotted",label="STANDARD DEVIATION")
    plt.legend(loc="upper right")
    
print ("Plotting the distributions of features in dataset")


### Scatter plot showing skewness and kurtosis of the IPP and DM-SNR Curve related to target_class

In [ ]:
# Scatter plotting skewness and kurtosis for Integrate Pulse Profile and
# the DM-SNR Curve related to the target_class
plt.figure(figsize=(14,7))
plt.subplot(121)
plt.scatter(x = " Excess kurtosis of the integrated profile",
            y = " Skewness of the integrated profile",
            data=data[data["target_class"] == 1],alpha=.7,
            label="pulsar stars",s=30,color = "g",linewidths=.4,edgecolors="black")
plt.scatter(x = " Excess kurtosis of the integrated profile",
            y = " Skewness of the integrated profile",
            data=data[data["target_class"] == 0],alpha=.6,
            label="not pulsar stars",s=30,color ="r",linewidths=.4,edgecolors="black")
plt.axvline(data[data["target_class"] == 1][" Skewness of the integrated profile"].mean(),
            color = "g",linestyle="dashed",label="mean pulsar star")
plt.axvline(data[data["target_class"] == 0][" Skewness of the integrated profile"].mean(),
            color = "r",linestyle="dashed",label ="mean non pulsar star")
plt.axhline(data[data["target_class"] == 1][" Skewness of the integrated profile"].mean(),
            color = "g",linestyle="dashed")
plt.axhline(data[data["target_class"] == 0][" Skewness of the integrated profile"].mean(),
            color = "r",linestyle="dashed")
plt.legend(loc ="best")
plt.xlabel("Kurtosis of IPP")
plt.ylabel("Skewness of IPP")
plt.title("Scatter plot for skewness and kurtosis or IPP for target class")
plt.subplot(122)
plt.scatter(x = " Skewness of the DM-SNR curve",y = ' Excess kurtosis of the DM-SNR curve',
            data=data[data["target_class"] == 0],alpha=.7,
            label="not pulsar stars",s=30,color ="r",linewidths=.4,edgecolors="black")
plt.scatter(x = " Skewness of the DM-SNR curve",y = ' Excess kurtosis of the DM-SNR curve',
            data=data[data["target_class"] == 1],alpha=.7,
            label="pulsar stars",s=30,color = "g",linewidths=.4,edgecolors="black")
plt.axvline(data[data["target_class"] == 1][" Excess kurtosis of the DM-SNR curve"].mean(),
            color = "g",linestyle="dashed",label ="mean pulsar star")
plt.axvline(data[data["target_class"] == 0][" Excess kurtosis of the DM-SNR curve"].mean(),
            color = "r",linestyle="dashed",label ="mean non pulsar star")
plt.axhline(data[data["target_class"] == 1][" Skewness of the DM-SNR curve"].mean(),
            color = "g",linestyle="dashed")
plt.axhline(data[data["target_class"] == 0][" Skewness of the DM-SNR curve"].mean(),
            color = "r",linestyle="dashed")
plt.legend(loc ="best")
plt.xlabel("Skewness of DM-SNR Curve")
plt.ylabel('Kurtosis of DM-SNR Curve')
plt.title("Scatter plot for skewness and kurtosis of DM-SNR Curve for target class")
plt.subplots_adjust(wspace =.4)

### Boxplots for attributes related to target class


In [ ]:
columns = [x for x in data.columns if x not in ["target_class"]]
length  = len(columns)
plt.figure(figsize=(13,20))
for i,j in itertools.zip_longest(columns,range(length)):
    plt.subplot(4,2,j+1)
    sns.lvplot(x=data["target_class"],y=data[i],palette=["red","blue"])
    plt.title(i)
    plt.subplots_adjust(hspace=.3)
    plt.axhline(data[i].mean(),linestyle = "dashed",color ="k",label ="Mean value for data")
    plt.legend(loc="best")
    
print ("Boxplots for features related to target_class")

### Histogram normality check and confirmation of absense of categorical features

In [ ]:
hist_cols = [' Mean of the integrated profile', 
            ' Standard deviation of the integrated profile', 
            ' Excess kurtosis of the integrated profile', 
            ' Skewness of the integrated profile',
            ' Mean of the DM-SNR curve', 
            ' Standard deviation of the DM-SNR curve', 
            ' Excess kurtosis of the DM-SNR curve',
            ' Skewness of the DM-SNR curve']

# Define figure size & axis
fig,ax = plt.subplots(figsize = (20,16))

# Plot histograms
data.hist(column=hist_cols,ax=ax);

### EDA Summary
By looking at the target class counts bar chart, we can clearly see there is a 
large class imbalance. True positives of our target class instance only make
up 9% of the dataset. 

The two features of kurtosis and skewness of the integrated pulse profile are 
closely correlated, in the early stages of the modeling section, I may end up removing
one of them. I will reexamine that after doing some baseline modeling and begin
doing feature selection and engineering.   
The mean and standard deviation  of the DM-SNR curve attributes also seem to be
correlated, but again, that will be revisited in the early stages of modeling. 
Also, the kurtosis and skewness of the DM-SNR curve also seem to be correlated. 
Finally, the kurtosis of the integrated pulse profile seems to be fairly correlated(.79)
with our target_class. I'll continue to test this correlation and see if there is a 
true, solid relationship between them.   

By comparing the means between our attributes to the target class, we can see some
possible relationships in the pointplots. It seems that the higher the mean of the 
integrated pulse profile and the skewness of the the DM-SNR curve, the less likely
the candidate stars are to be actual pulsars.   
When we look at the standard deviation of the two classes compared to one another, 
the higher the skewness of the DM-SNR curve also seems to suggest a higher likelihood
of being a false positive.   

None of the features follow a true normal distribution. Kurtosis and skewness of the IPP 
are close to normal, but both are ever so slightly positively skewed.  
As mentioned before, I will model both the raw data and transformed & normalized data 
to see if there is an increase in accuracy from normalizing these attributes.   

When we examine the scatterplots that display the relationships between the skewness
and kurtosis of the IPP and DM-SNR curves when related to the target_class, we can 
make the following assumptions:   
The higher the kurtosis and skewness of the IPP, the more likely it is that the candidate star is a pulsar.   
The lower kurtosis and skewness of the DM-SNR curve, the more likely it is that the candidate star is a pulsar.   

By looking at the boxplots of the attributes, here are some brief summaries of what is shown:   
- The mean of the IPP appears to be higher for false positives.    
- The standared deviation of the IPP for the false positives appear to be slightly higher and have more upper end outliers.   
- The kurtosis of the IPP seems to be smaller/close to 0 for false positives.   
- The skewness of the IPP again is around 0 for false positives, while for actual pulsars, it is much larger.   
- The mean of the DM-SNR curve is typically higher for actual pulsars, compared to false positives.   
- The standard deviation of the DM-SNR curve is higher for actual pulsars than false positives.  
- The kurtosis of the DM-SNR curve is higher for false positives than for actual pulsars.   
- The skewness of the DM-SNR curve is significantly higher for false positives than for actual pulsars.    

Finally, we can see again in the histograms that there are no normal distributions and 
none of our data is categorical. 

# Model the Data
In this section I'll dive into the pre-processing related to modeling, normalize the secondary dataset, account for class imbalance, run a baseline model to be improved upon, and use a variety of libraries to create a number of types of models to see what performs best in the context. "Best" is going to be related to the model accuracy, using F1 score(especially for target class) as a metric of success. We'll take a close look at confusion matrices for each model type, and review all types in a conclusion.

### Transformation and Normalization 
Below I'll normalize our cloned dataset to be modeled alongside with and compared to our raw dataset. 

In [ ]:
# check the preview of the dataset 
TNdata.head()

In [ ]:
# use sklearn's preprocessing MinMaxScaler
x = TNdata.values # returns a numpy array
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
TNdata = pd.DataFrame(x_scaled, columns=TNdata.columns)

In [ ]:
TNdata.head()

In [ ]:
# confirm the transformation didn't affect our target class
TNdata.target_class.value_counts()

### Test and Train Splits
Now I'll separate out the target class from the predictor variables and do a train-test split for both sets of data, and account for XGBoost. I've opted to do a 75/25 split.

In [ ]:
# define X and y of our datasets
y_TN = TNdata['target_class']
X_TN = TNdata.drop(['target_class'], axis=1)

y = data['target_class']
X = data.drop(['target_class'], axis=1)

# define random seed
seed = 25

# split data into train and test sets
X_train_TN, X_test_TN, y_train_TN, y_test_TN = train_test_split(X_TN, 
                                                    y_TN, 
                                                    test_size=0.25, 
                                                    random_state=seed)

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.25, 
                                                    random_state=seed)

# split in numpy array format for XGBoost
X_train_xgb_TN, X_test_xgb_TN, y_train_xgb_TN, y_test_xgb_TN = train_test_split(X_TN.values, 
                                                                    y_TN.values, 
                                                                    test_size=0.25, 
                                                                    random_state=seed)

X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(X.values, 
                                                                    y.values, 
                                                                    test_size=0.25, 
                                                                    random_state=seed)

### Class Imbalance
I'm going to use SMOTE(Synthetic Minority Oversampling TEchnique) to oversample the target class.  
It it important to only use SMOTE on the training data as to not disturb the test set.  

In [ ]:
# Previous original class distribution
print("Before:")
print(f"\n{y_train.value_counts()}")
print(f"\n{pd.Series(y_train_xgb).value_counts()}")

# Fit SMOTE to training data
smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_sample(X_train, y_train)
X_train_resampled_TN, y_train_resampled_TN = smote.fit_sample(X_train_TN, y_train_TN)

# format for XGBoost
X_train_resampled_xgb, y_train_resampled_xgb = smote.fit_sample(X_train_xgb, y_train_xgb)
X_train_resampled_xgb_TN, y_train_resampled_xgb_TN = smote.fit_sample(X_train_xgb_TN, y_train_xgb_TN)

# Preview synthetic sample class distribution
print("\nAfter:\n")
print(f"{pd.Series(y_train_resampled).value_counts()}") 
print(f"\n{pd.Series(y_train_resampled_xgb).value_counts()}")

### Baseline Model
For the baseline model, I'll use the Naive Bayes Classification Algorithm. The Naive Bayes models are very fast and simple algorithms often used for high-dimensional datasets. I'm opting to use this algorithm as a baseline model due to its speed, few tunable parameters, and its reputation as a "quick-and-dirty" baseline for classification problems.   
In order to create a true baseline, I'll only be using the raw dataset for this, and not using the normalized set.  

### Naive Bayes Classification

In [ ]:
aggs = data.groupby('target_class').agg(['mean', 'std'])
aggs

In [ ]:
# calculating conditional probability point estimates
def p_x_given_class(obs_row, feature, class_):
    mu = aggs[feature]['mean'][class_]
    std = aggs[feature]['std'][class_]
    
    # A single observation
    obs = data.iloc[obs_row][feature] 
    
    p_x_given_y = stats.norm.pdf(obs, loc=mu, scale=std)
    return p_x_given_y

In [ ]:
# testing function with one feature
p_x_given_class(0, ' Mean of the integrated profile', 0)

In [ ]:
# multinomial Bayes using all the features
row = 1
c_probs = []
for c in range(2):
    # Initialize probability to relative probability of class 
    p = len(data[data['target_class'] == c])/len(data) 
    for feature in X.columns:
        p *= p_x_given_class(row, feature, c) 
        # Update the probability using the point estimate for each feature
        c_probs.append(p)

c_probs

In [ ]:
# calculating class probabilities for obs
def predict_class(row):
    c_probs = []
    for c in range(2):
        # Initialize probability to relative probability of class
        p = len(data[data['target_class'] == c])/len(data) 
        for feature in X.columns:
            p *= p_x_given_class(row, feature, c)
        c_probs.append(p)
    return np.argmax(c_probs)

In [ ]:
predict_class(row)

In [ ]:
# apply prediction function to the train and test sets
y_hat_train = [predict_class(X_train.iloc[idx]) for idx in range(len(X_train))]
y_hat_test = [predict_class(X_test.iloc[idx]) for idx in range(len(X_test))]

In [ ]:
# calculate training and test accuracy
residuals_train = y_hat_train == y_train
acc_train = residuals_train.sum()/len(residuals_train)

residuals_test = y_hat_test == y_test
acc_test = residuals_test.sum()/len(residuals_test)
print('Training Accuracy: {}\tTesting Accuracy: {}'.format(acc_train, acc_test))

#### Baseline model summary
As expected, the accuracy of this model is pretty abysmal. It gives us a good starting place to work from and shows how nuanced our dataset is and how dependant on multiple variables the target class is. 

### Create Classifiers - Definining Critical Functions
Below I'll define a few functions that will be used in this final modeling phase, instantiate classifiers, run the various models, and then compare the results.  
The models to be run are as follows:  
- Gaussian Naive Bayes
- Decision Tree
- Bagging Tree 
- K Nearest Neighbor
- Random Forest 
- Logistic Regression
- Extra Tree 
- Gradient Boost
- AdaBoost
- XGBoost 

#### Important Functions

In [ ]:
# Define function to batch classifiers

def batch_clf(X_train, y_train, X_test, y_test, clf_dict, verbose = True):
    '''
    Fits a dictionary of classifiers, makes predictions, plots ROC, returns metrics
    
    Args:
        X_train: {array-like, sparse matrix} of shape (n_samples, n_features) train input values
        y_train: array-like of shape (n_samples,) train target values
        X_test: {array-like, sparse matrix} of shape (m_samples, m_features) test input values
        y_test: array-like of shape (m_samples,) test target values
        clf_dict: dictionary with key name of classifier and value classifier instance
        verbose: if True, prints time taken to fit and predict for each classifier
        
    Returns:
        Results dataframe
    '''
    # Create empty DataFrame to store results
    times = []
    train_acc_scores = []
    test_acc_scores = []
    train_f1_scores = []
    test_f1_scores = []
    train_precision_scores = []
    test_precision_scores = []
    train_recall_scores = []
    test_recall_scores = []
    train_roc_data = []
    test_roc_data = []
    test_profit_scores = []
    
    # Loop through dictionary items
    for key, clf in clf_dict.items():
        start_time = time.clock()
        
        # Fit classifier
        clf_fitted = clf.fit(X_train,y_train)
        
        # Get Predictions
        train_preds = clf_fitted.predict(X_train)
        test_preds = clf_fitted.predict(X_test)
  
        
        #Get accuracy scores
        train_acc = accuracy_score(y_train, train_preds)
        train_acc_scores.append(round(train_acc,2))
        test_acc = accuracy_score(y_test, test_preds)
        test_acc_scores.append(round(test_acc,2))
        
        #Get F1 Scores
        train_f1 = f1_score(y_train, train_preds)
        train_f1_scores.append(round(train_f1,2))
        test_f1 = f1_score(y_test, test_preds)
        test_f1_scores.append(round(test_f1,2))
        
        # Get Precision Scores
        train_precision = precision_score(y_train, train_preds)
        train_precision_scores.append(round(train_precision,2))
        test_precision = precision_score(y_test, test_preds)
        test_precision_scores.append(round(test_precision,2))
        
        # Get Recall Scores
        train_recall = recall_score(y_train, train_preds)
        train_recall_scores.append(round(train_recall,2))
        test_recall = recall_score(y_test, test_preds)
        test_recall_scores.append(round(test_recall,2))
        
        # Get Probability Predictions
        train_hat = clf_fitted.predict_proba(X_train)
        train_proba = train_hat[:,1]
        fpr_train, tpr_train, thresholds_train = roc_curve(y_train, train_proba)
        train_roc_data.append([fpr_train, tpr_train, thresholds_train])
            
        test_hat = clf_fitted.predict_proba(X_test)
        test_proba = test_hat[:,1]
        fpr_test, tpr_test, thresholds_test = roc_curve(y_test, test_proba)
        test_roc_data.append([fpr_test, tpr_test, thresholds_test])
        
        # Get Profit
        
        #best_t, best_profit = profit_threshold_optimizer(y_test, test_proba)
        #test_profit_scores.append(best_profit)

        end_time = time.clock()
        time_elapsed = end_time - start_time
        times.append(round(time_elapsed,2))
        
        if verbose:
            print(f'trained {key} in {round(time_elapsed,2)}')
        
    # Create results dataframe
    results = pd.DataFrame({'Model': list(clf_dict.keys()), 
                            'Time': times,
                            'Train Accuracy': train_acc_scores,
                            'Test Accuracy': test_acc_scores, 
                            'Train F1': train_f1_scores,
                            'Test F1': test_f1_scores,
                            'Train Precision' : train_precision_scores,
                            'Test Precision' : test_precision_scores,
                            'Train Recall': train_recall_scores,
                            'Test Recall': test_recall_scores,
                            'Test Profit' : test_profit_scores
                            })
    
   # Plot side by side ROC curve
    fig, axes = plt.subplots(1,2, figsize = (13,6))
    
    for i in range(len(train_roc_data)):
        axes[0].plot(train_roc_data[i][0], train_roc_data[i][1], lw=4, \
                 label= f'{list(clf_dict.keys())[i]}')
        
    for i in range(len(test_roc_data)):
        axes[1].plot(test_roc_data[i][0], test_roc_data[i][1], lw=4, \
                 label= f'{list(clf_dict.keys())[i]}')    
        
    for ax in axes:
        ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        ax.set_xlabel('False Positive Rate')
        ax.set_ylabel('True Positive Rate')
        ax.legend(loc='lower right')
    axes[0].set_title('Receiver operating characteristic (ROC) Curve \n Training Set')
    axes[1].set_title('Receiver operating characteristic (ROC) Curve \n Test Set')
    plt.show()

    return results

In [ ]:
# Define function to fit our models and predict on the training and test sets

def clf_pred(models):
    '''
    Intakes our DataFrame with our models 
        and our X & y, train & test data
    
    Returns a DataFrame with all of our models, 
        metrics and accuracy scores   
    '''
    
    # Initialize matrix to fill
    clf_df = np.zeros((len(models), 26), dtype=object)
    
    for i,model in models.iterrows():
        

        # Classifier Names & Models
        clf_df[i,0] = model['clf_name']
        clf_df[i,1] = model['clfs'] 
        
        ### Assign Variables ###
        # We do this here because XGBoost takes 
        # np.array format unlike the other models
        X_train = model['X_train']
        X_test = model['X_test']
        y_train = model['y_train']
        y_test = model['y_test']

        ### Initialize Timer ### 
        start_time = time.time()
        
        ### Fit & Predict ###
    
        # Fit Model
        fitted = model['clfs'].fit(X_train, y_train)
        clf_df[i,2] = fitted
        
        # Calculate time to fit model
        stop_time = time.time()
        runtime = (stop_time - start_time)
        clf_df[i,3] = runtime
    
        # Predict
        y_pred_train = fitted.predict(X_train)
        y_pred_test = fitted.predict(X_test)

        clf_df[i,4] = y_pred_train
        clf_df[i,5] = y_pred_test
    
        # y_score
        y_score_train = fitted.predict_proba(X_train)
        y_score_test = fitted.predict_proba(X_test)
        clf_df[i,6] = y_score_train
        clf_df[i,7] = y_score_test
    
        # False & True Positive Rates
        clf_df[i,8], clf_df[i,9], thresholds_train = roc_curve(y_train, y_score_train[:,1])
        clf_df[i,10], clf_df[i,11], thresholds_test = roc_curve(y_test, y_score_test[:,1])
        
        
        ### Accuracy Scores ### 
        
        # Precision
        clf_df[i,12] = precision_score(y_train, y_pred_train)
        clf_df[i,13] = precision_score(y_test, y_pred_test)
        
        # Recall 
        clf_df[i,14] = recall_score(y_train, y_pred_train)
        clf_df[i,15] = recall_score(y_test, y_pred_test)        
        
        # F1
        clf_df[i,16] = f1_score(y_train, y_pred_train)
        clf_df[i,17] = f1_score(y_test, y_pred_test)  
        
        # Accuracy
        clf_df[i,18] = accuracy_score(y_train, y_pred_train)
        clf_df[i,19] = accuracy_score(y_test, y_pred_test)
        
        # AUC
        clf_df[i,20] = roc_auc_score(y_train, y_pred_train)
        clf_df[i,21] = roc_auc_score(y_test, y_pred_test)
        
        ### Add X & y values to have everything in one place ### 
        # These are class balanced/resampled #
        clf_df[i,22] = X_train
        clf_df[i,23] = X_test
        clf_df[i,24] = y_train
        clf_df[i,25] = y_test        
    
    ### Create DataFrame ###
    
    # Column Names
    columns = ['Classifier',
               'Model',
               'Fitted Model',
               'Runtime',
               'Train Preds',
               'Test Preds',
               'Train y-Score',
               'Test y-Score',
               'Train FPR',
               'Train TPR',
               'Test FPR',
               'Test TPR',
               'Train Precision',
               'Test Precision',
               'Train Recall',
               'Test Recall',
               'Train F1',
               'Test F1',
               'Train Accuracy',
               'Test Accuracy',
               'Train ROC AUC',
               'Test ROC AUC',
               'X_train',
               'X_test',
               'y_train',
               'y_test'
              ]
    
    # Create DataFrame
    clf_df = pd.DataFrame(clf_df, columns=columns)
    
    return clf_df

In [ ]:
# Function to combine classification report, confusion matrix, 
# accuracy score, roc curve, and auc in one easy way

def model(algorithm, X_train, y_train, X_test, y_test, of_type):
    algorithm.fit(X_train, y_train)
    preds = algorithm.predict(X_test)
    
    print(algorithm)
    print("\naccuracy_score:",accuracy_score(y_test, preds))
    print("\nclassification report:\n", (classification_report(y_test, preds)))
    
    plt.figure(figsize=(14,10))
    plt.subplot(221)
    ax = sns.heatmap(confusion_matrix(y_test, preds), annot=True, fmt="d", 
                linecolor="k", linewidth=2)
    # work around for top & bottom cutoff is pasted below
    bottom, top = ax.get_ylim()
    ax.set_ylim(bottom + 0.5, top - 0.5)
    plt.title("Confusion Matrix", fontsize=20)
    
    predicting_probabilities = algorithm.predict_proba(X_test)[:,1]
    fpr, tpr, thresholds = roc_curve(y_test, predicting_probabilities)
    plt.subplot(222)
    plt.plot(fpr, tpr, label = ("Area Under the Curve:", auc(fpr, tpr)), color="r")
    plt.plot([1,0], [1,0], linestyle = "dashed", color="k")
    plt.legend(loc="best")
    plt.title("ROC Curve and Area Under Curve", fontsize=20)
    
    if of_type == "feat":
        dataframe = pd.DataFrame(algorithm.feature_importances_, X_train.columns).reset_index()
        dataframe = dataframe.rename(columns={"index":"features", 0:"coefficients"})
        dataframe = dataframe.sort_values(by="coefficients", ascending=False)
        # plot conf_matrix, roc/auc, and feat importances
        plt.subplot(223)
        ax = sns.barplot(x = "coefficients", y = "features", data = dataframe, palette = "husl")
        plt.title("Feature Importances", fontsize=20)
        for i, j in enumerate(dataframe["coefficients"]):
            ax.text(.011, i, j, weight = "bold")
        
    elif of_type == "coef":
        dataframe = pd.DataFrame(algorithm.coef_.ravel(), X_train.columns).reset_index()
        dataframe = dataframe.rename(columns={"index":"features", 0:"coefficients"})
        dataframe = dataframe.sort_values(by="coefficients", ascending=False)
        # plot conf_matrix, roc/auc, and feat importances
        plt.subplot(223)
        ax = sns.barplot(x = "coefficients", y = "features", data = dataframe, palette = "husl")
        plt.title("Feature Importances", fontsize=20)
        for i, j in enumerate(dataframe["coefficients"]):
            ax.text(.011, i, j, weight = "bold")
            
    elif of_type == "none":
        return(algorithm)

#### Create Classifiers

In [ ]:
# Gaussian Naive Bayes
gnb = GaussianNB() # none

# Decision Tree
dt = DecisionTreeClassifier() # feat

# Bagging Tree 
bag = BaggingClassifier(base_estimator=DecisionTreeClassifier(),
                        random_state=seed) # feat

# K Nearest Neighbor
knn = KNeighborsClassifier() # none

# Random Forest 
rf = RandomForestClassifier() # feat

# Logistic Regression
log = LogisticRegression() # coef

# Extra Tree 
ext = ExtraTreeClassifier()  # feat

# Gradient Boost
gb = GradientBoostingClassifier() # feat

# AdaBoost
adb = AdaBoostClassifier() # feat

# XGBoost 
xgb = xgb.XGBClassifier(random_state=seed)

# Create dictionary of classifiers for raw data
clf_dict = {0:
    {'clf_name': 'Gaussian Naive Bayes',
     'clfs': gnb,
     'X_train': X_train_resampled,
     'X_test': X_test,
     'y_train': y_train_resampled,
     'y_test': y_test},
            1:
    {'clf_name': 'Decision Tree',
     'clfs': dt,
     'X_train': X_train_resampled,
     'X_test': X_test,
     'y_train': y_train_resampled,
     'y_test': y_test},
            2:
    {'clf_name': 'Bagging Tree',
     'clfs': bag,
     'X_train': X_train_resampled,
     'X_test': X_test_xgb,
     'y_train': y_train_resampled,
     'y_test': y_test_xgb},
            3:
    {'clf_name': 'K Nearest Neighbor',
     'clfs': knn,
     'X_train': X_train_resampled,
     'X_test': X_test,
     'y_train': y_train_resampled,
     'y_test': y_test},
           4:
    {'clf_name': 'Random Forest',
     'clfs': rf,
     'X_train': X_train_resampled,
     'X_test': X_test,
     'y_train': y_train_resampled,
     'y_test': y_test},
           5:
    {'clf_name': 'Logistic Regression',
     'clfs': log,
     'X_train': X_train_resampled,
     'X_test': X_test,
     'y_train': y_train_resampled,
     'y_test': y_test},
           6:
    {'clf_name': 'Extra Tree',
     'clfs': ext,
     'X_train': X_train_resampled,
     'X_test': X_test,
     'y_train': y_train_resampled,
     'y_test': y_test},
           7:
    {'clf_name': 'Gradient Boost',
     'clfs': gb,
     'X_train': X_train_resampled,
     'X_test': X_test,
     'y_train': y_train_resampled,
     'y_test': y_test},
           8:
    {'clf_name': 'AdaBoost',
     'clfs': adb,
     'X_train': X_train_resampled,
     'X_test': X_test,
     'y_train': y_train_resampled,
     'y_test': y_test},
           9:
    {'clf_name': 'XGBoost',
     'clfs': xgb,
     'X_train': X_train_resampled_xgb,
     'X_test': X_test_xgb,
     'y_train': y_train_resampled_xgb,
     'y_test': y_test_xgb}}

# Convert into DataFrame

models_df = pd.DataFrame.from_dict(clf_dict, orient='index')

# Create dictionary of classifiers for normalized data
clf_dict_TN = {0:
    {'clf_name': 'Gaussian Naive Bayes',
     'clfs': gnb,
     'X_train': X_train_resampled_TN,
     'X_test': X_test,
     'y_train': y_train_resampled_TN,
     'y_test': y_test},
            1:
    {'clf_name': 'Decision Tree',
     'clfs': dt,
     'X_train': X_train_resampled_TN,
     'X_test': X_test,
     'y_train': y_train_resampled_TN,
     'y_test': y_test},
            2:
    {'clf_name': 'Bagging Tree',
     'clfs': bag,
     'X_train': X_train_resampled_TN,
     'X_test': X_test_xgb,
     'y_train': y_train_resampled_TN,
     'y_test': y_test_xgb},
            3:
    {'clf_name': 'K Nearest Neighbor',
     'clfs': knn,
     'X_train': X_train_resampled_TN,
     'X_test': X_test,
     'y_train': y_train_resampled_TN,
     'y_test': y_test},
           4:
    {'clf_name': 'Random Forest',
     'clfs': rf,
     'X_train': X_train_resampled_TN,
     'X_test': X_test,
     'y_train': y_train_resampled_TN,
     'y_test': y_test},
           5:
    {'clf_name': 'Logistic Regression',
     'clfs': log,
     'X_train': X_train_resampled_TN,
     'X_test': X_test,
     'y_train': y_train_resampled_TN,
     'y_test': y_test},
           6:
    {'clf_name': 'Extra Tree',
     'clfs': ext,
     'X_train': X_train_resampled_TN,
     'X_test': X_test,
     'y_train': y_train_resampled_TN,
     'y_test': y_test},
           7:
    {'clf_name': 'Gradient Boost',
     'clfs': gb,
     'X_train': X_train_resampled_TN,
     'X_test': X_test,
     'y_train': y_train_resampled_TN,
     'y_test': y_test},
           8:
    {'clf_name': 'AdaBoost',
     'clfs': adb,
     'X_train': X_train_resampled_TN,
     'X_test': X_test,
     'y_train': y_train_resampled_TN,
     'y_test': y_test},
           9:
    {'clf_name': 'XGBoost',
     'clfs': xgb,
     'X_train': X_train_resampled_xgb_TN,
     'X_test': X_test,
     'y_train': y_train_resampled_xgb_TN,
     'y_test': y_test}}

# Convert into DataFrame

models_df_TN = pd.DataFrame.from_dict(clf_dict_TN, orient='index')

### Fit and Predict
I'll use the above classifier predictions function to run through both dictionaries of Classifiers. The function organizes the results on a table for easy comparison. 

#### Prediction on raw data

In [ ]:
clf_preds = clf_pred(models_df)
clf_preds

### Gaussian Naive Bayes

In [ ]:
# raw data
model(gnb, X_train, y_train, X_test, y_test, "none")

In [ ]:
# normalized data
model(gnb, X_train_TN, y_train_TN, X_test, y_test, "none")

The non-normalized data vastly outperformed the normalized data in the Gaussian Naive Bayes model with an accuracy score of 95 vs 74.  
A target F1 score of .75, this model performed moderately overall. An AUC of .95 indicates decent performance. 

### Decision Tree

In [ ]:
# raw data
model(dt, X_train, y_train, X_test, y_test, "feat")

In [ ]:
# normalized data
model(dt, X_train_TN, y_train_TN, X_test, y_test, "feat")

Again, the normalized data performed horribly - a .12 F1 score for the target class.    
However, for our raw data the model performed well. With an overall accuracy of .97 and a target class F1 of .82, it performed better than out Gaussian Naive Bayes model. The AUC of .90 indicates the model performed well.   
We can see that the overwhelmingly most important feature is the Kurtosis of the Integrated Pulse Profile.  
There were far fewer false positives and a similar level of false negatives as the GNB. 

### Bagging Tree

In [ ]:
# raw data
model(bag, X_train, y_train, X_test, y_test, "none")

In [ ]:
# normalized data
model(bag, X_train_TN, y_train_TN, X_test, y_test, "none")

This seems to be becoming a trend...The normalized data performed terribly - a .01 F1 score for the target class.    
Our raw data the model performed well. An overall accuracy of .98 and a target class F1 of .88 is good, with very few false positives and less false negatives than GNB or Decision Tree models. The high AUC of .95 supports that. 

### K Nearest Neighbor

In [ ]:
# raw data
model(knn, X_train, y_train, X_test, y_test, "none")

In [ ]:
# normalized data
model(knn, X_train_TN, y_train_TN, X_test, y_test, "none")

Again, normalized model can be ignored.  
An overall accuracy of .97 and a target F1 score of .85 is good, supported by the low number of false positives and comparable(to GNB, Dec Tree, Bagging Tree) false negatives and high AUC of .93.

### Random Forest

In [ ]:
# raw data
model(rf, X_train, y_train, X_test, y_test, "feat")

In [ ]:
# normalized data
model(rf, X_train_TN, y_train_TN, X_test, y_test, "feat")

With an accuracy score of .98, a target class F1 score of .87, only 30 false positives, 73 false negatives, and an AUC of .96 this model is our best performing so far. As you can see by looking at the feature importance graph, it seems to take the most amount of features into account which makes me feel a bit more confident in this algorithm and its ability to deal with this type of dataset. 

### Logistic Regression

In [ ]:
# raw data
model(log, X_train, y_train, X_test, y_test, "coef")

In [ ]:
# normalized data
model(log, X_train_TN, y_train_TN, X_test, y_test, "coef")

This model performed very similarly to our Random Forest model in most metrics. 

### Extra Tree

In [ ]:
# raw data
model(ext, X_train, y_train, X_test, y_test, "feat")

In [ ]:
# normalized data
model(ext, X_train_TN, y_train_TN, X_test, y_test, "feat")

The Extra Tree model performed similarly to the Decision tree model, almost the exact same metrics.  
Even though the normalized model is for all intents and purposes, useless, it's interesting to see the feature importances graph topped by the mean IPP feature. 

### Gradient Boost

In [ ]:
# raw data
model(gb, X_train, y_train, X_test, y_test, "feat")

In [ ]:
# normalized data
model(gb, X_train_TN, y_train_TN, X_test, y_test, "feat")

The raw data Gradient Boost model is so far the best performing model with an accuracy score of .98, a macro avg F1 of .93, a target class F1 of .88, the lowest number of false positives and negatives out of the models run thusfar, and a AUC of .97.

### AdaBoost

In [ ]:
# raw
model(adb, X_train, y_train, X_test, y_test, "feat")

In [ ]:
# normalized data
model(adb, X_train_TN, y_train_TN, X_test, y_test, "feat")

The raw data AdaBoost model performed on level with the Gradient Boost model, which is quite well. Almost identical metrics.   

### XGBoost

In [ ]:
# testing with non-normalized data
clf = XGBClassifier(random_state=25)
clf.fit(X_train, y_train)
training_preds = clf.predict(X_train)
val_preds = clf.predict(X_test)
training_accuracy = accuracy_score(y_train, training_preds)
val_accuracy = accuracy_score(y_test, val_preds)

print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))

In [ ]:
xgbo = XGBClassifier(random_state=seed)

In [ ]:
# confusion matrix for XGB
model(xgbo, X_train, y_train, X_test, y_test, "feat")

### Summary of Models
Due to their horrible levels of performance, the models built off of the normalized dataset will be ignored entirely for this comparison. The table below describes the performance of the models:


| Classifier | Accuracy Score | Macro Avg. F1 Score | Target Class F1 score | False Pos | False Neg | AUC | Runtime(sec) |
| --- | --- | --- | --- | --- | --- | --- | --- |
| XGBoost | 99 | .93 | .87 | 30 | 72 | .97 | 0.673 |
| AdaBoost | 98 | .93 | .87 | 29 | 74 | .97 | 1.230 |
| Bagging Tree | 98 | .93 | .88 | 29 | 71 | .95 | 1.237 |
| Random Forest | 98 | .93 | .87 | 30 | 73 | .96 | 3.447 |
| Gradient Boost | 98 | .93 | .88 | 38 | 64 | .97 | 5.597 |
| Logistic Regression | 98 | .93 | .87 | 27 | 75 | .97 | 0.124 |
| Decision Tree | 97 | .90 | .82 | 70 | 77 | .90 | 0.221 |
| KNN | 97 | .92 | .85 | 38 | 81 | .93 | 0.043 |
| Extra Tree | 97 | .90 | .82 | 78 | 74 | .90 | 0.010 |
| Gassian Naive Bayes | 95 | .86 | .75 | 161 | 69 | .95 | 0.006 |


## Tuning Model Hyperparameters  
In this stage, I take my top 3 best performing models: XGBoost, AdaBoost, and Bagging Tree, and look to increase their performance via GridSearch to find the ideal parameters. 

#### GridSearch optimization for XGBoost

In [ ]:
clf = XGBClassifier(random_state=25)

param_grid = {
    'max_depth': [20, 50, 100, 200],
    'learning_rate': [0.125, 0.25, 0.5, 1],
    'subsample': [.6, .75, .9],
    'min_child_weight': [10, 20, 30],
    'n_estimators': [10, 100, 500]
}
gridsearch_xgb = GridSearchCV(clf, param_grid, scoring='accuracy', cv=3)
gridsearch_xgb.fit(X_train, y_train)
print(f"Training Accuracy: {gridsearch_xgb.best_score_ :.2%}")
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))
print("")
print(f"Optimal Parameters: {gridsearch_xgb.best_params_}")

In [ ]:
xgboTuned = XGBClassifier(random_state=seed, learning_rate=0.25, 
                              max_depth=20, min_child_weight=20,
                              n_estimators=100, subsample=0.9)

In [ ]:
model(xgboTuned, X_train, y_train, X_test, y_test, "feat")

#### GridSearch optimization for AdaBoost

In [ ]:
clf = AdaBoostClassifier()

param_grid = {
    'n_estimators': [20, 50, 100, 200],
    'learning_rate': [0.125, 0.25, 0.5, 1],
}
gridsearch_adb = GridSearchCV(clf, param_grid, scoring='accuracy', cv=3)
gridsearch_adb.fit(X_train, y_train)
print(f"Training Accuracy: {gridsearch_adb.best_score_ :.2%}")
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))
print("")
print(f"Optimal Parameters: {gridsearch_adb.best_params_}")

In [ ]:
adbTuned = AdaBoostClassifier(learning_rate=0.25, n_estimators=200)

In [ ]:
model(adbTuned, X_train, y_train, X_test, y_test, "feat")

#### GridSearch optimization for Bagging Tree

In [ ]:
clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(),
                        random_state=seed)

param_grid = {
    'bootstrap': [True, False],
    'bootstrap_features': [True, False],
    'n_estimators': [10, 50, 100],
    'max_samples' : [10, 100],
    'base_estimator__criterion': ['gini','entropy'],
    'base_estimator__max_features' : [0.6, 0.8, 1.0],
    'base_estimator__min_samples_leaf': [1, 3, 5, 10]
}
gridsearch_bag = GridSearchCV(clf, param_grid, scoring='accuracy', cv=3)
gridsearch_bag.fit(X_train, y_train)
print(f"Training Accuracy: {gridsearch_bag.best_score_ :.2%}")
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))
print("")
print(f"Optimal Parameters: {gridsearch_bag.best_params_}")

In [ ]:
bagTuned = BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                                  max_features=1,
                                                                  min_samples_leaf=1,
                                                                  random_state=seed),
                             bootstrap=True, bootstrap_features=False,
                             max_samples=100, n_estimators=100, random_state=seed)

In [ ]:
model(bagTuned, X_train, y_train, X_test, y_test, "none")

# Interpret the Data - Conclusion

Here in the final step, I examine the findings developed by modeling and indentify important insights. As more data becomes available, this cycle can repeat and continue in a loop to build a faster, less biased, and more accurate model. For now, I'll review the models completed with the data available.  

### Model Performance (Top 3 models, comparing metrics)

These findings are presented in the following format: (tuned) base. All models scored a 97.72% Validation accuracy, supporting their resistance to overtraining. 

| Classifier | Accuracy Score | Macro Avg. F1 Score | Target Class F1 score | False Pos | False Neg | AUC | Runtime(sec) |
| --- | --- | --- | --- | --- | --- | --- | --- |
| XGBoost | (98)99 | (.93).93 | (.88).87 | (33)30 | (67)72 | (.97).97 | 0.673 |
| AdaBoost | (98)98 | (.93).93 | (.86).87 | (31)29 | (77)74 | (.97).97 | 1.230 |
| Bagging Tree | (97)98 | (.91).93 | (.83).88 | (20)29 | (107)71 | (.96).95 | 1.237 |

The three best performing classifiers were XGBoost, AdaBoost, and Bagging Tree. They all were able to produce high accuracy, high validation test scores, and low numbers of type I and type II errors. XGBoost performed narrowly better than the latter two, with a higher accuracy, macro average F1 score, Target class F1 score, and AUC. All of the top 3 models scored a 97.72 in validation score, showing their resistance to overtraining. Also, the top 3 models all had similar type I and type II errors.  

Coming from a baseline model with a <20% accuracy, there was significant improvement made by dealing with class imbalances, using more robust algorithms, and tuning those algorithms to the ideal parameters. 

XGBoost is the ideal model, based off of accuracy, validation, and runtime. 


### Feature Importance

By far, excess kurtosis of the Integrated Pulse Profile was the most important feature for classifying these candidate stars.  

By examining the feature importance charts, we can see how impactful it was in the XGBoost and Adaboost models. The Bagging Tree classifier seems to be less one dimensional in that regard and shows a fairly high importance for Standard Deviation of the DM-SNR curve, and the Mean of the IPP.  


### EDA Summary -  Confirming Suspicions

Now to revisit my post-EDA impressions below:
By looking at the target class counts bar chart, we can clearly see there is a 
large class imbalance. True positives of our target class instance only make
up 9% of the dataset. 

The two features of kurtosis and skewness of the integrated pulse profile are 
closely correlated, in the early stages of the modeling section, I may end up removing
one of them. I will reexamine that after doing some baseline modeling and begin
doing feature selection and engineering.   
The mean and standard deviation  of the DM-SNR curve attributes also seem to be
correlated, but again, that will be revisited in the early stages of modeling. 
Also, the kurtosis and skewness of the DM-SNR curve also seem to be correlated. 
Finally, the kurtosis of the integrated pulse profile seems to be fairly correlated(.79)
with our target_class. I'll continue to test this correlation and see if there is a 
true, solid relationship between them.   

By comparing the means between our attributes to the target class, we can see some
possible relationships in the pointplots. It seems that the higher the mean of the 
integrated pulse profile and the skewness of the the DM-SNR curve, the less likely
the candidate stars are to be actual pulsars.   
When we look at the standard deviation of the two classes compared to one another, 
the higher the skewness of the DM-SNR curve also seems to suggest a higher likelihood
of being a false positive.   

None of the features follow a true normal distribution. Kurtosis and skewness of the IPP 
are close to normal, but both are ever so slightly positively skewed.  
As mentioned before, I will model both the raw data and transformed & normalized data 
to see if there is an increase in accuracy from normalizing these attributes.   

When we examine the scatterplots that display the relationships between the skewness
and kurtosis of the IPP and DM-SNR curves when related to the target_class, we can 
make the following assumptions:   
The higher the kurtosis and skewness of the IPP, the more likely it is that the candidate star is a pulsar.   
The lower kurtosis and skewness of the DM-SNR curve, the more likely it is that the candidate star is a pulsar.   

By looking at the boxplots of the attributes, here are some brief summaries of what is shown:   
- The mean of the IPP appears to be higher for false positives.    
- The standared deviation of the IPP for the false positives appear to be slightly higher and have more upper end outliers.   
- The kurtosis of the IPP seems to be smaller/close to 0 for false positives.   
- The skewness of the IPP again is around 0 for false positives, while for actual pulsars, it is much larger.   
- The mean of the DM-SNR curve is typically higher for actual pulsars, compared to false positives.   
- The standard deviation of the DM-SNR curve is higher for actual pulsars than false positives.  
- The kurtosis of the DM-SNR curve is higher for false positives than for actual pulsars.   
- The skewness of the DM-SNR curve is significantly higher for false positives than for actual pulsars.    

Finally, we can see again in the histograms that there are no normal distributions and 
none of our data is categorical. 

As I had expected, the kurtosis of the IPP was strongly related to confirming a True Positive. 

### Final Thoughts - Recommendations and Potential Future Work

#### Model Selection
I recommend using XGBoost. It is a faster, more robust model, capable of high levels of accuracy and validation. 

#### Business Value
Overall, our model(s) performed well and could prove to be valuable assets to the field. With the ability to quickly and reliably classify candidate stars, we can more easily identify pulsars to study further. 

#### Larger Dataset
For future work, I would find it beneficial to have a larger dataset, with more anomolies in it to be able to refine our model even further. Looking at other forms of radio interference and how to train our telescopes to avoid misclassification of these candidate stars would help "trim the fat" off of a larger dataset and effectively pre-process for us. 

#### More Computational Power
It would be beneficial in the case of a larger dataset to be able to take advantage of a supercomputer or a distributed system to be able to efficiently process massive datasets. These things are more available in the research scenarios and professional observational astronomy labs. 

#### More Complex ML / DL
I am curious to see if more advanced machine learning & deep learning techniques would yeild better results. 

### References

Special thanks to Kuo Liu of the Max Planck Institute for allowing me to cite and use his work on pulsars. 

[1] Kuo Liu (2017). "Introduction to Pulsar, Pulsar Timing, and measuring of Pulse Time-of-Arrivals", Max-Planck-Institut für Radioastronomie, Bonn, Germany

